In [5]:
"""Example of a simple guard scheduling problem."""
from ortools.sat.python import cp_model


def main():
    # Data.
    num_guards = 6
    num_shifts = 3
    num_days = 7
    all_guards = range(num_guards)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_guards:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one guard in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_guards) == 1)

    # Each guard works at most one shift per day.
    for n in all_guards:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

    # Try to distribute the shifts evenly, so that each guard works
    # min_shifts_per_guard shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of guards, some guards will
    # be assigned one more shift.
    min_shifts_per_guard = (num_shifts * num_days) // num_guards
    if num_shifts * num_days % num_guards == 0:
        max_shifts_per_guard = min_shifts_per_guard
    else:
        max_shifts_per_guard = min_shifts_per_guard + 1
    for n in all_guards:
        num_shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked.append(shifts[(n, d, s)])
        model.Add(min_shifts_per_guard <= sum(num_shifts_worked))
        model.Add(sum(num_shifts_worked) <= max_shifts_per_guard)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True


    class GuardsPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_guards, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_guards = num_guards
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print('Solution %i' % self._solution_count)
            for d in range(self._num_days):
                print('Day %i' % d)
                for n in range(self._num_guards):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.Value(self._shifts[(n, d, s)]):
                            is_working = True
                            print('  Guard %i works shift %i' % (n, s))
                    if not is_working:
                        print('  Guard {} does not work'.format(n))
            if self._solution_count >= self._solution_limit:
                print('Stop search after %i solutions' % self._solution_limit)
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = GuardsPartialSolutionPrinter(shifts, num_guards,
                                                    num_days, num_shifts,
                                                    solution_limit)

    solver.Solve(model, solution_printer)

    # Statistics.
    print('\nStatistics')
    print('  - conflicts      : %i' % solver.NumConflicts())
    print('  - branches       : %i' % solver.NumBranches())
    print('  - wall time      : %f s' % solver.WallTime())
    print('  - solutions found: %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Solution 1
Day 0
  Guard 0 does not work
  Guard 1 does not work
  Guard 2 does not work
  Guard 3 works shift 1
  Guard 4 works shift 2
  Guard 5 works shift 0
Day 1
  Guard 0 does not work
  Guard 1 does not work
  Guard 2 does not work
  Guard 3 works shift 0
  Guard 4 works shift 1
  Guard 5 works shift 2
Day 2
  Guard 0 does not work
  Guard 1 does not work
  Guard 2 does not work
  Guard 3 works shift 2
  Guard 4 works shift 1
  Guard 5 works shift 0
Day 3
  Guard 0 works shift 2
  Guard 1 works shift 1
  Guard 2 works shift 0
  Guard 3 does not work
  Guard 4 does not work
  Guard 5 does not work
Day 4
  Guard 0 works shift 1
  Guard 1 works shift 0
  Guard 2 works shift 2
  Guard 3 does not work
  Guard 4 does not work
  Guard 5 does not work
Day 5
  Guard 0 works shift 1
  Guard 1 works shift 2
  Guard 2 works shift 0
  Guard 3 does not work
  Guard 4 does not work
  Guard 5 does not work
Day 6
  Guard 0 works shift 2
  Guard 1 works shift 1
  Guard 2 works shift 0
  Guard 3 d